In [1]:
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def calc_co_occurrence(mat, dist=(0,1), max_val=255, symmetric=False, norm=False):
    """
    Calculate the Co-Occurrence matrix for a given image and distance vector.

    Parameters:
    mat (numpy array): Input image matrix.
    dist (tuple): Distance vector (dx, dy).
    max_val (int): Maximum pixel value in the image.
    symmetric (bool): If True, make the matrix symmetric.
    norm (bool): If True, normalize the matrix.

    Returns:
    numpy array: Co-Occurrence matrix.
    """
    co_occurrence_mat = np.zeros((max_val, max_val), dtype=np.int32)

    rows, cols = mat.shape
    for i in range(rows):
        for j in range(cols):
            current_pixel = mat[i, j]
            neighbor_i, neighbor_j = i + dist[1], j + dist[0]
            if 0 <= neighbor_i < rows and 0 <= neighbor_j < cols:
                neighbor_pixel = mat[neighbor_i, neighbor_j]
                co_occurrence_mat[current_pixel, neighbor_pixel] += 1
                if symmetric:
                    co_occurrence_mat[neighbor_pixel, current_pixel] += 1

    if symmetric:
        co_occurrence_mat = co_occurrence_mat + co_occurrence_mat.T - np.diag(co_occurrence_mat.diagonal())

    if norm:
        total = co_occurrence_mat.sum()
        if total != 0:
            co_occurrence_mat = co_occurrence_mat / total

    return co_occurrence_mat

def get_energy(cm):
    """Calculate the energy of the Co-Occurrence matrix."""
    return np.sum(cm ** 2)

def get_entropy(cm):
    """Calculate the entropy of the Co-Occurrence matrix."""
    with np.errstate(divide='ignore', invalid='ignore'):
        entropy = -np.sum(cm * np.log2(cm + (cm == 0)))
    return entropy

def get_contrast(cm):
    """Calculate the contrast of the Co-Occurrence matrix."""
    rows, cols = cm.shape
    contrast = 0
    for i in range(rows):
        for j in range(cols):
            contrast += (i - j) ** 2 * cm[i, j]
    return contrast

def get_homogeneity(cm):
    """Calculate the homogeneity of the Co-Occurrence matrix."""
    rows, cols = cm.shape
    homogeneity = 0
    for i in range(rows):
        for j in range(cols):
            homogeneity += cm[i, j] / (1 + abs(i - j))
    return homogeneity


if __name__ == "__main__":
    # Set fixed seed for reproducible results
    # np.random.seed(0)

    # # Generate a sample image with values between [0, max_val)
    max_val = 5
    # mat = np.random.randint(low=0, high=max_val, size=(5,5), dtype=np.uint8)

    # print('Eingabe:\n')
    # print(mat)
    mat = np.array([[3, 4, 2, 4, 4],
                    [1, 2, 2, 2, 4],
                    [3, 2, 4, 1, 3],
                    [1, 3, 4, 0, 3],
                    [1, 4, 3, 0, 0]])
    # mat = np.array([[0, 0, 1, 2],
    #                 [3, 3, 0, 3],
    #                 [3, 1, 1, 1],
    #                 [3, 2, 1, 0]])
    

    cm = calc_co_occurrence(mat, dist=(2, 0), max_val=max_val, symmetric=False, norm=False)

    print('\nCo-Occurrence Matrix:\n')
    print(cm)

    # Using skimage's greycomatrix for comparison
    glcm = graycomatrix(mat, [2], [0], levels=max_val, symmetric=False, normed=False)

    print('GLCM:\n')
    print(glcm.squeeze())

    # Calculate texture properties
    energy = get_energy(cm)
    entropy = get_entropy(cm)
    contrast = get_contrast(cm)
    homogeneity = get_homogeneity(cm)

    print(f'\nEnergy: {energy}')
    print(f'Entropy: {entropy}')
    print(f'Contrast: {contrast}')
    print(f'Homogeneity: {homogeneity}')


Current: 3, Neighbor: 2
Current: 4, Neighbor: 4
Current: 2, Neighbor: 4
Current: 1, Neighbor: 2
Current: 2, Neighbor: 2
Current: 2, Neighbor: 4
Current: 3, Neighbor: 4
Current: 2, Neighbor: 1
Current: 4, Neighbor: 3
Current: 1, Neighbor: 4
Current: 3, Neighbor: 0
Current: 4, Neighbor: 3
Current: 1, Neighbor: 3
Current: 4, Neighbor: 0
Current: 3, Neighbor: 0

Co-Occurrence Matrix:

[[0 0 0 0 0]
 [0 0 1 1 1]
 [0 1 1 0 2]
 [2 0 1 0 1]
 [1 0 0 2 1]]
GLCM:

[[0 0 0 0 0]
 [0 0 1 1 1]
 [0 1 1 0 2]
 [2 0 1 0 1]
 [1 0 0 2 1]]

Energy: 21
Entropy: -6.0
Contrast: 61
Homogeneity: 6.95


In [2]:
cm = calc_co_occurrence(mat, dist=(1, 1), max_val=max_val, symmetric=False, norm=False)

print('\nCo-Occurrence Matrix:\n')
print(cm)

# Using skimage's greycomatrix for comparison
glcm = graycomatrix(mat, [1], [1], levels=max_val, symmetric=False, normed=False)

print('GLCM:\n')
print(glcm.squeeze())

Current: 3, Neighbor: 2
Current: 4, Neighbor: 2
Current: 2, Neighbor: 2
Current: 4, Neighbor: 4
Current: 1, Neighbor: 2
Current: 2, Neighbor: 4
Current: 2, Neighbor: 1
Current: 2, Neighbor: 3
Current: 3, Neighbor: 3
Current: 2, Neighbor: 4
Current: 4, Neighbor: 0
Current: 1, Neighbor: 3
Current: 1, Neighbor: 4
Current: 3, Neighbor: 3
Current: 4, Neighbor: 0
Current: 0, Neighbor: 0

Co-Occurrence Matrix:

[[1 0 0 0 0]
 [0 0 1 1 1]
 [0 1 1 1 2]
 [0 0 1 2 0]
 [2 0 1 0 1]]
GLCM:

[[1 0 0 0 0]
 [0 0 1 1 1]
 [0 1 1 1 2]
 [0 0 1 2 0]
 [2 0 1 0 1]]


In [3]:
mat = np.array([[3, 4, 2, 4, 4],
                [1, 2, 2, 2, 4],
                [3, 2, 4, 1, 3],
                [1, 3, 4, 0, 3],
                [1, 4, 3, 0, 0]])

# Calculate the Co-Occurrence matrix with normalization
cm = calc_co_occurrence(mat, dist=(2, 0), max_val=5, symmetric=False, norm=True)

print('Co-Occurrence Matrix:\n', cm)

# Calculate texture properties using custom functions
energy = get_energy(cm)
entropy = get_entropy(cm)
contrast = get_contrast(cm)
homogeneity = get_homogeneity(cm)

print(f'\nCustom Calculations:')
print(f'Energy: {energy}')
print(f'Entropy: {entropy}')
print(f'Contrast: {contrast}')
print(f'Homogeneity: {homogeneity}')

# Calculate Co-Occurrence matrices using skimage's graycomatrix
glcm = graycomatrix(mat, [2], [0], levels=5, symmetric=False, normed=True)

print('\nGLCM:\n', glcm.squeeze())

# Calculate texture properties using skimage's graycoprops
sk_energy = graycoprops(glcm, 'energy')[0, 0]
sk_contrast = graycoprops(glcm, 'contrast')[0, 0]
sk_homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

print(f'\nskimage Calculations:')
print(f'Energy: {sk_energy}')
print(f'Contrast: {sk_contrast}')
print(f'Homogeneity: {sk_homogeneity}')

Current: 3, Neighbor: 2
Current: 4, Neighbor: 4
Current: 2, Neighbor: 4
Current: 1, Neighbor: 2
Current: 2, Neighbor: 2
Current: 2, Neighbor: 4
Current: 3, Neighbor: 4
Current: 2, Neighbor: 1
Current: 4, Neighbor: 3
Current: 1, Neighbor: 4
Current: 3, Neighbor: 0
Current: 4, Neighbor: 3
Current: 1, Neighbor: 3
Current: 4, Neighbor: 0
Current: 3, Neighbor: 0
Co-Occurrence Matrix:
 [[0.         0.         0.         0.         0.        ]
 [0.         0.         0.06666667 0.06666667 0.06666667]
 [0.         0.06666667 0.06666667 0.         0.13333333]
 [0.13333333 0.         0.06666667 0.         0.06666667]
 [0.06666667 0.         0.         0.13333333 0.06666667]]

Custom Calculations:
Energy: 0.09333333333333332
Entropy: 3.506890595608519
Contrast: 4.066666666666667
Homogeneity: 0.46333333333333326

GLCM:
 [[0.         0.         0.         0.         0.        ]
 [0.         0.         0.06666667 0.06666667 0.06666667]
 [0.         0.06666667 0.06666667 0.         0.13333333]
 [0.13